# Test of GAT
- use DGL
- predict `graphs`
- test: 0~99
- validation: 100~199
- train: 200~999
- larger lr with scheduler
- try the sklearn report

In [16]:
import os
import dgl
import json
import torch
import torch as th
# from tqdm import tqdm
from tqdm.notebook import tqdm  # 使用 notebook 版本的 tqdm
import torch.nn as nn
from dgl.nn import GraphConv, GATConv
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.metrics import classification_report


- check the GPU and assign the GPU by the best memory usage

In [17]:
import subprocess
import torch

def get_free_gpu():
    try:
        # Run nvidia-smi command to get GPU details
        _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
        command = "nvidia-smi --query-gpu=memory.free --format=csv,nounits,noheader"
        memory_free_info = _output_to_list(subprocess.check_output(command.split())) 
        memory_free_values = [int(x) for i, x in enumerate(memory_free_info)]
        
        # Get the GPU with the maximum free memory
        best_gpu_id = memory_free_values.index(max(memory_free_values))
        return best_gpu_id
    except:
        # If any exception occurs, default to GPU 0 (this handles cases where nvidia-smi isn't installed)
        return 0

if torch.cuda.is_available():
    # Get the best GPU ID based on free memory and set it
    best_gpu_id = get_free_gpu()
    device = torch.device(f"cuda:{best_gpu_id}")
else:
    device = torch.device("cpu")
    print("there's no available GPU")

# device = torch.device(f"cuda:1")
print(device)


cuda:1


## Fix the seed

In [18]:
import numpy as np
import torch
import random

#fix seed
def same_seeds(seed = 8787):
    torch.manual_seed(seed)
    # random.seed(seed) 
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## Data Loader

In [19]:
class GraphDataset(Dataset):
    def __init__(self, data_list, device):
        self.data_list = data_list
        self.device = device

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        data = self.data_list[idx]

        g = dgl.graph((th.tensor(data["edge_index"][0]), th.tensor(data["edge_index"][1])), num_nodes=data["num_nodes"]).to(self.device)

        g.ndata['feat'] = th.tensor(data["node_feat"]).to(self.device)
        g.edata['feat'] = th.tensor(data["edge_attr"]).to(self.device)  # Add edge features to graph

        return g, th.tensor(data["label"]).to(self.device)


def collate(samples):
    # The input `samples` is a list of pairs
    #  (graph, label).
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.tensor(labels)


In [20]:
datasets = ['train', 'valid', 'test']
# datasets = ['test']
dataset_data = {}

for dataset_name in tqdm(datasets):
#     file_path = f"../../data_processing/dgl/data/test_graph/repeated_{dataset_name}.jsonl"
    file_path = f"../../data_processing/dgl/data_new/training_data/exp_2/transH_150/{dataset_name}.jsonl"
    
    print(file_path)
    with open(file_path) as f:
        data_list = [json.loads(line) for line in tqdm(f, position=0, leave=True)]
    
    dataset_data[dataset_name] = GraphDataset(data_list, device)

print("Datasets loaded!")

  0%|          | 0/3 [00:00<?, ?it/s]

../../data_processing/dgl/data_new/training_data/exp_2/transH_150/train.jsonl


0it [00:00, ?it/s]

../../data_processing/dgl/data_new/training_data/exp_2/transH_150/valid.jsonl


0it [00:00, ?it/s]

../../data_processing/dgl/data_new/training_data/exp_2/transH_150/test.jsonl


0it [00:00, ?it/s]

Datasets loaded!


- choose batch size

In [21]:
def create_dataloaders(batch_size, shuffle=True):
    dataloaders = {}
    for dataset_name, dataset in dataset_data.items():
        # do not shuffle the testing dataset
        if dataset_name == "test":
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)    
        else:
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate)
    return dataloaders

# dataloaders = create_dataloaders(4)
dataloaders = create_dataloaders(16)


if (len(dataloaders['test'].dataset) + len(dataloaders['valid'].dataset) + len(dataloaders['train'].dataset)) % 165 != 0: print("Error data!!")
else: print("OK!")


Error data!!


- Turn the print message to a log file

In [22]:
# print(dataloaders['test'][5])
sample = dataset_data['train'][5000]
print(sample)

print(len(dataloaders['test'].dataset))
print(len(dataloaders['valid'].dataset))
print(len(dataloaders['train'].dataset))
print(len(dataloaders['test'].dataset) + len(dataloaders['valid'].dataset) + len(dataloaders['train'].dataset))

(Graph(num_nodes=10, num_edges=19,
      ndata_schemes={'feat': Scheme(shape=(150,), dtype=torch.float32)}
      edata_schemes={'feat': Scheme(shape=(150,), dtype=torch.float32)}), tensor(6, device='cuda:1'))
16600
16600
132800
166000


In [23]:
import datetime

now = datetime.datetime.now()

formatted_time = now.strftime("%m%d_%H:%M")

log_file_path = f"../log_message/{formatted_time}_GAT_transH_150.log"

def add_log_msg(msg, log_file_path=log_file_path):
    with open(log_file_path, 'a') as f:
        f.write(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}\n')
    print(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}')

print(log_file_path)

../log_message/0902_12:41_GAT_transH_150.log


### Model

In [24]:
class GAT(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_heads, dropout_prob=0.25):
        super(GAT, self).__init__()
        
        # do not check the zero in_degree since we have all the complete graph
        self.layer1 = GATConv(in_dim, hidden_dim, num_heads=num_heads, activation=F.relu, allow_zero_in_degree=True)
        self.layer2 = GATConv(hidden_dim * num_heads, out_dim, num_heads=num_heads, allow_zero_in_degree=True)
        
        # Adding Batch Normalization after each GAT layer
        self.batchnorm1 = nn.BatchNorm1d(hidden_dim * num_heads)
        self.batchnorm2 = nn.BatchNorm1d(out_dim)
        
        # Adding Dropout for regularization
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, g, h):
        # Apply GAT layers
        h = self.layer1(g, h)
        h = h.view(h.shape[0], -1)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.layer2(g, h).squeeze(1)
        
        # Store the output as a new node feature
        g.ndata['h_out'] = h

        # Use mean pooling to aggregate this new node feature
        h_agg = dgl.mean_nodes(g, feat='h_out')
        return h_agg

    

- Model Forward  

In [25]:
def model_fn(data, model, criterion, device, count=1, which_type='train'):
    """Forward a batch through the model."""
    batched_g, labels = data
    batched_g = batched_g.to(device)
    
    labels = labels.to(device)
    logits = model(batched_g, batched_g.ndata['feat'].float()) # for GAT
    logits = logits.mean(dim=1)
    
    loss = criterion(logits, labels)

    # Get the class id with the highest probability
    preds = logits.argmax(1)
    
    # Compute accuracy
    accuracy = torch.mean((preds == labels).float())
    
    if which_type == 'validation' and count % 1000 == 0:
        add_log_msg(f"labels of Validation: {labels} {labels.shape}")
        add_log_msg(f"predicted of Validation: {preds} {preds.shape}")
        
    elif which_type == 'test'  and count % 1000 == 0:
        add_log_msg(f"labels of Test: {labels} {labels.shape}")
        add_log_msg(f"predicted of Test: {preds} {preds.shape}")
        
    if count % 5000 == 0: 
        add_log_msg(f"labels of {count}: {labels} {labels.shape}")
        add_log_msg(f"predicted of {count}: {preds} {preds.shape}")
        
    return loss, accuracy, preds

### Training

- Fix the seed and save the model.state_dict that contains the initial weight

In [26]:
seed = 8787
same_seeds(seed)

model = GAT(in_dim=150, hidden_dim=16, out_dim=168, num_heads=8)
torch.save(model.state_dict(), 'model1_initial/initial_weight.pth')

In [27]:
model.layer1.fc.weight

Parameter containing:
tensor([[-0.1088, -0.0760,  0.1288,  ..., -0.1192, -0.0175,  0.0021],
        [ 0.0325,  0.0195, -0.1396,  ...,  0.0864, -0.0855,  0.1670],
        [-0.0041, -0.0149, -0.0657,  ..., -0.0180, -0.0733,  0.1429],
        ...,
        [ 0.2402, -0.1547, -0.0973,  ...,  0.0845, -0.2335, -0.0765],
        [ 0.1358,  0.1819, -0.1881,  ..., -0.0790, -0.0578, -0.2301],
        [ 0.0297, -0.0572, -0.1424,  ...,  0.2224, -0.1195,  0.0005]],
       requires_grad=True)

- Check if model really load the model_dict

In [28]:
model = GAT(in_dim=150, hidden_dim=16, out_dim=168, num_heads=8)
model.load_state_dict(torch.load('model1_initial/initial_weight.pth'))
model.layer1.fc.weight

Parameter containing:
tensor([[-0.1088, -0.0760,  0.1288,  ..., -0.1192, -0.0175,  0.0021],
        [ 0.0325,  0.0195, -0.1396,  ...,  0.0864, -0.0855,  0.1670],
        [-0.0041, -0.0149, -0.0657,  ..., -0.0180, -0.0733,  0.1429],
        ...,
        [ 0.2402, -0.1547, -0.0973,  ...,  0.0845, -0.2335, -0.0765],
        [ 0.1358,  0.1819, -0.1881,  ..., -0.0790, -0.0578, -0.2301],
        [ 0.0297, -0.0572, -0.1424,  ...,  0.2224, -0.1195,  0.0005]],
       requires_grad=True)

### test of valid and test part is ``graph``

- Batch size = 4
- use large lr and scheduler

In [ ]:
import csv
import pandas as pd
from sklearn.metrics import classification_report
from torch.optim import AdamW, lr_scheduler

seed = 8787
same_seeds(seed)

model = GAT(in_dim=150, hidden_dim=16, out_dim=168, num_heads=8)
# in_dim means the dimension of the node_feat(50 dim, since the 50-dim embedding)
# out_dim means the # of the categories -> 168 for out tasks
model.load_state_dict(torch.load('model1_initial/initial_weight.pth'))
best_model_path = "../checkpoint_GAT/best_model_GAT_transH_150.pt"

model = model.to(device)

# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
optimizer = AdamW(model.parameters(), lr=5e-4)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=18, num_training_steps=total_steps)

# T_max control the period of the lr changing -> set 1/10 first
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=36, eta_min=0, last_epoch=- 1, verbose=False)


criterion = nn.CrossEntropyLoss()
total_steps = 25

# save the best model
best_val_loss = float('inf')
patience = 5  # Number of epochs with no improvement after which training will be stopped.
waiting = 0  # The number of epochs with no improvement so far.


# Training Part
for epoch in tqdm(range(total_steps)):
    # Train
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
    
    for data in tqdm(dataloaders['train'], desc="Training", position=0, leave=True):
        num_batches += 1
        loss, accuracy, _ = model_fn(data, model, criterion, device, num_batches, which_type='train')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()

        
#     scheduler.step()
    add_log_msg(f"total batches: {num_batches}")

    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches

    add_log_msg(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')

    
    # Validation Part
    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0
    num_batches = 0


    with torch.no_grad():
        for data in tqdm(dataloaders['valid'], desc="Validation", position=0, leave=True):
            loss, accuracy, _ = model_fn(data, model, criterion, device, num_batches, which_type='validation')
            total_accuracy += accuracy.item()
            total_loss += loss.item()
            num_batches += 1

    avg_accuracy = total_accuracy / num_batches
    current_loss = total_loss / num_batches
    
    add_log_msg(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}\n')
    
            
    if current_loss < best_val_loss:
        best_val_loss = current_loss
        waiting = 0
        
        if os.path.exists(best_model_path):
            os.remove(best_model_path)
            add_log_msg("Find a better model!!")

        torch.save(model.state_dict(), best_model_path)

        
#         print(best_model_path)

    else:
        waiting += 1
        if waiting >= patience:
            add_log_msg("============================== Early stopping ==================================")
            break

  0%|          | 0/25 [00:00<?, ?it/s]

Training:   0%|          | 0/8300 [00:00<?, ?it/s]

09/01/2023, 22:19:17# labels of 5000: tensor([ 85, 134,  50,   0, 122,  30,   5, 125,  63,  55, 154,  89,  95,  64,
        141,  12], device='cuda:0') torch.Size([16])
09/01/2023, 22:19:17# predicted of 5000: tensor([ 17, 134,  17,  17,  70, 161,  17, 161, 161, 161, 154,  17,  17,  28,
        161, 161], device='cuda:0') torch.Size([16])
09/01/2023, 22:20:27# total batches: 8300
09/01/2023, 22:20:27# Epoch 0 | Train Loss: 4.6406 | Train Accuracy: 0.0490


Validation:   0%|          | 0/1038 [00:00<?, ?it/s]

09/01/2023, 22:20:27# labels of Validation: tensor([ 84,  99,   9,   4,  77, 115,  34,  84,  74, 115,  28,  93,  19, 141,
         98,  86], device='cuda:0') torch.Size([16])
09/01/2023, 22:20:27# predicted of Validation: tensor([128,  99, 132, 132, 132, 146, 132, 128, 132, 146, 132, 132, 132, 132,
        132, 132], device='cuda:0') torch.Size([16])
09/01/2023, 22:20:27# labels of 0: tensor([ 84,  99,   9,   4,  77, 115,  34,  84,  74, 115,  28,  93,  19, 141,
         98,  86], device='cuda:0') torch.Size([16])
09/01/2023, 22:20:27# predicted of 0: tensor([128,  99, 132, 132, 132, 146, 132, 128, 132, 146, 132, 132, 132, 132,
        132, 132], device='cuda:0') torch.Size([16])
09/01/2023, 22:20:46# labels of Validation: tensor([125, 107, 155, 148, 117,  23,  20, 110,  72, 111,  28,  47, 166,  60,
        130,  82], device='cuda:0') torch.Size([16])
09/01/2023, 22:20:46# predicted of Validation: tensor([132, 132, 132, 132,  40,  99, 132, 132, 146, 132, 132, 132, 132, 132,
        127,

...

Training:   0%|          | 0/8300 [00:00<?, ?it/s]

09/01/2023, 23:10:13# labels of 5000: tensor([ 60,  82,  74,   6,  30, 162, 155,  98,  85,  84,  69,  77, 118, 121,
         79,  52], device='cuda:0') torch.Size([16])
09/01/2023, 23:10:13# predicted of 5000: tensor([158,  82, 109, 132, 158, 109,  28,  60,  85,  84,  69,  37, 118, 104,
         72,  52], device='cuda:0') torch.Size([16])
09/01/2023, 23:11:31# total batches: 8300
09/01/2023, 23:11:31# Epoch 14 | Train Loss: 2.9552 | Train Accuracy: 0.3402


Validation:   0%|          | 0/1038 [00:00<?, ?it/s]

09/01/2023, 23:11:32# labels of Validation: tensor([ 23,  98,   9,  71,  54,  50, 152,  73, 164, 104,   5,   1,  34, 151,
        127,  71], device='cuda:0') torch.Size([16])
09/01/2023, 23:11:32# predicted of Validation: tensor([ 23,  75,  75,  75,  75,  50,  75,  73,  75,  75,   5,  75,  75,  75,
        127,  75], device='cuda:0') torch.Size([16])
09/01/2023, 23:11:32# labels of 0: tensor([ 23,  98,   9,  71,  54,  50, 152,  73, 164, 104,   5,   1,  34, 151,
        127,  71], device='cuda:0') torch.Size([16])
09/01/2023, 23:11:32# predicted of 0: tensor([ 23,  75,  75,  75,  75,  50,  75,  73,  75,  75,   5,  75,  75,  75,
        127,  75], device='cuda:0') torch.Size([16])
09/01/2023, 23:11:51# labels of Validation: tensor([142,  96,  76,  46,  65,  19,  45, 153,  52,  56, 106, 103, 109,  77,
        116,  83], device='cuda:0') torch.Size([16])
09/01/2023, 23:11:51# predicted of Validation: tensor([ 75,  96,  75,  75,  75,  75,   5,  75,  52,  56, 106,   6,  75,   6,
         75,

Training:   0%|          | 0/8300 [00:00<?, ?it/s]

### Testing Part

In [29]:
# load the pretrained model
pretrained_model_path = '../checkpoint_GAT/best_model_GAT_transH_150.pt'
model.load_state_dict(torch.load(pretrained_model_path))

model.to(device)
model.eval()

total = 0
correct = 0
count = 0

true_labels = []
predicted_labels = []

with torch.no_grad():
    for data in tqdm(dataloaders['test'], desc="Testing", position=0, leave=True):

        loss, accuracy, predicted = model_fn(data, model, criterion, device, count, which_type='test')
        labels = data[1].to(device)
        
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())
        
        if count % 5000 == 0:
            add_log_msg(f"labels: {labels} {labels.shape}")
            add_log_msg(f"predicted: {predicted} {predicted.shape}")
            
        count += 1
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

add_log_msg(f'Test Accuracy: {100 * correct / total} %\n\n\n')


# ======================================== handlig the output excel files ========================================
mapping_file = './new_mapping.txt'
label_mapping = {}
with open(mapping_file, 'r') as f:
    for line in f:
        parts = line.strip().split(': ')
        label_mapping[int(parts[1])] = parts[0]
        
# 将映射后的标签应用到true和predicted标签列表
mapped_true_labels = [label_mapping[label] for label in true_labels]
mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

# 生成Scikit-learn报告信息的DataFrame
report_data = classification_report(mapped_true_labels, mapped_predicted_labels, output_dict=True)
report_df = pd.DataFrame(report_data).transpose()

report_folder = 'classification_report'
os.makedirs(report_folder, exist_ok=True)

count = 0
while True:
    report_filename = f'classification_report-transH_150-{count}.xlsx'
    labels_filename = f'mapped_true_predicted_labels-transH_150-{count}.xlsx'
    
    report_path = os.path.join(report_folder, report_filename)
    labels_path = os.path.join(report_folder, labels_filename)
    
    if not os.path.exists(report_path) and not os.path.exists(labels_path):
        break
    count += 1

    
report_df.to_excel(report_path, index_label='Label')

mapped_labels_df = pd.DataFrame({'true_label': mapped_true_labels, 'predicted_label': mapped_predicted_labels})
mapped_labels_df.to_excel(labels_path, index=False)

add_log_msg(f"report path: {report_path}")
add_log_msg(f"label path: {labels_path}")

mapped_report = classification_report(mapped_true_labels, mapped_predicted_labels)
add_log_msg(f"mapped_report:\n{mapped_report}")

Testing:   0%|          | 0/1038 [00:00<?, ?it/s]

09/02/2023, 12:41:27# labels of Test: tensor([65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       device='cuda:1') torch.Size([16])
09/02/2023, 12:41:27# predicted of Test: tensor([43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43],
       device='cuda:1') torch.Size([16])
09/02/2023, 12:41:27# labels of 0: tensor([65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       device='cuda:1') torch.Size([16])
09/02/2023, 12:41:27# predicted of 0: tensor([43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43],
       device='cuda:1') torch.Size([16])
09/02/2023, 12:41:27# labels: tensor([65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65],
       device='cuda:1') torch.Size([16])
09/02/2023, 12:41:27# predicted: tensor([43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43],
       device='cuda:1') torch.Size([16])
09/02/2023, 12:41:43# labels of Test: tensor([162, 162, 162, 162, 162, 162, 162, 162, 162, 162, 162, 

/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


09/02/2023, 12:42:08# report path: classification_report/classification_report-transH_150-0.xlsx
09/02/2023, 12:42:08# label path: classification_report/mapped_true_predicted_labels-transH_150-0.xlsx


/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


09/02/2023, 12:42:08# mapped_report:
                                                precision    recall  f1-score   support

T1003.001_0ef4cc7b-611c-4237-b20b-db36b6906554       1.00      1.00      1.00       100
    T1003.001_35d92515122effdd73801c6ac3021da7       0.93      0.99      0.96       100
    T1003.002_5a484b65c247675e3b7ada4ba648d376       0.00      0.00      0.00       100
    T1003.002_7fa4ea18694f2552547b65e23952cabb       0.99      1.00      1.00       100
    T1003.003_9f73269695e54311dd61dc68940fb3e1       0.00      0.00      0.00       100
    T1003.003_f049b89533298c2d6cd37a940248b219       0.00      0.00      0.00       100
        T1003_18f31c311ac208802e88ab8d5af8603e       0.90      0.98      0.94       100
        T1007_9d03c91bdae5a80f17f89c987942b5a8       1.00      0.99      0.99       100
    T1007_c6607391-d02c-44b5-9b13-d3492ca58599       0.00      0.00      0.00       100
        T1007_d6bb2a19da7246731ed9c44831b135f8       0.00      0.00      0.00     

/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
